In [ ]:
import pandas as pd
import numpy as np
import pickle
from flask import Flask, request, jsonify
from sklearn.preprocessing import StandardScaler

: 

In [ ]:
# Carregar modelos treinados
with open("knn_model.pkl", "rb") as knn_file:
    knn = pickle.load(knn_file)
with open("scaler.pkl", "rb") as scaler_file:
    scaler = pickle.load(scaler_file)
with open("kmeans_model.pkl", "rb") as kmeans_file:
    kmeans = pickle.load(kmeans_file)
with open("history_mapping.pkl", "rb") as history_file:
    history_mapping = pickle.load(history_file)

# Criar Flask API
app = Flask(__name__)

# Carregar os dados
features_df = pd.read_excel("treino_feature.xlsx")
user_ids = features_df["userId"]
X = features_df.drop(columns=["userId"])
X_scaled = scaler.transform(X)

# Função para recomendar páginas tratando cold-start
def recomendar_paginas(user_id):
    if user_id not in history_mapping or len(history_mapping[user_id]) == 0:
        # Cold-start: Buscar recomendação baseada no cluster mais próximo
        user_features = np.zeros((1, X_scaled.shape[1]))  # Usuário sem histórico
        cluster_pred = knn.predict(user_features)[0]  # Encontrar cluster mais próximo
        
        # Encontrar usuários no mesmo cluster
        similar_users = features_df[features_df["cluster"] == cluster_pred]["userId"]
        recomendacoes = set()
        for similar_user in similar_users:
            recomendacoes.update(history_mapping.get(similar_user, []))
        
        return list(recomendacoes)[:6]  # Retorna sugestões baseadas no cluster
    
    user_index = user_ids[user_ids == user_id].index[0]
    user_features = X_scaled[user_index].reshape(1, -1)
    neighbors = knn.kneighbors(user_features, n_neighbors=6, return_distance=False)[0]
    
    recomendacoes = set()
    for neighbor in neighbors:
        neighbor_id = user_ids.iloc[neighbor]
        recomendacoes.update(history_mapping.get(neighbor_id, []))
    
    return list(recomendacoes)[:6]  # Retorna as top 6 páginas recomendadas dos usuários mais similares

# Criar endpoint para recomendação
@app.route("/recommend", methods=["GET"])
def recommend():
    user_id = request.args.get("user_id")
    if not user_id:
        return jsonify({"error": "User ID is required"}), 400
    
    recomendacoes = recomendar_paginas(user_id)
    return jsonify({"user_id": user_id, "recommendations": recomendacoes})

# Rodar a API
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000, debug=True)


: 